<a href="https://colab.research.google.com/github/KimW00Sung/fastapi-websocket-chat/blob/main/recording.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install fastapi uvicorn openai-whisper soundfile pyngrok python-multipart

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 14.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 851.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.7 MB/s eta

In [22]:
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
!tar -xvf ngrok-v3-stable-linux-amd64.tgz
!chmod +x ngrok

--2025-05-27 14:41:45--  https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 13.248.244.96, 99.83.220.108, 35.71.179.82, ...
Connecting to bin.equinox.io (bin.equinox.io)|13.248.244.96|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9395172 (9.0M) [application/octet-stream]
Saving to: ‘ngrok-v3-stable-linux-amd64.tgz.1’

ngrok-v3-stable-lin 100%[===================>]   8.96M  15.3MB/s    in 0.6s    

2025-05-27 14:41:46 (15.3 MB/s) - ‘ngrok-v3-stable-linux-amd64.tgz.1’ saved [9395172/9395172]

ngrok


In [23]:
# 여기에 본인의 AUTH_TOKEN을 입력하세요
AUTH_TOKEN = "2xNmD31mGpn4V4cbqJF7sBEpiPY_6JbQxP2rMPfaS3XTUX1Pg"
!./ngrok config add-authtoken {AUTH_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [24]:
%%writefile app.py
# FastAPI 서버 코드
from fastapi import FastAPI, Request, File, UploadFile
from fastapi.responses import HTMLResponse, JSONResponse
from fastapi.staticfiles import StaticFiles
from fastapi.templating import Jinja2Templates
from fastapi.middleware.cors import CORSMiddleware
import whisper
import io
import numpy as np
import soundfile as sf
import uvicorn

app = FastAPI()

# CORS 설정 추가
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # 모든 도메인에서 접근 허용
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# 정적 파일 및 템플릿 설정
app.mount("/static", StaticFiles(directory="www"), name="static")
templates = Jinja2Templates(directory="www")

# Whisper 모델 로드
model = whisper.load_model("base")

@app.get("/", response_class=HTMLResponse)
async def index(request: Request):
    return templates.TemplateResponse("index.html", {"request": request})

@app.post("/transcribe")
async def transcribe(file: UploadFile = File(...)):
    # 파일 데이터를 메모리에 직접 로드
    audio_data = await file.read()
    audio_stream = io.BytesIO(audio_data)

    try:
        # 오디오 데이터를 NumPy 배열로 변환
        audio_array, sample_rate = sf.read(audio_stream)

        # 단일 채널로 변환 (Whisper 요구사항)
        if len(audio_array.shape) > 1 and audio_array.shape[1] > 1:
            audio_array = audio_array.mean(axis=1)

        # Whisper 모델에 배열 직접 전달
        result = model.transcribe(audio_array)

        return JSONResponse(content={"text": result["text"]})
    except Exception as e:
        print(f'Error: {e}')
        return JSONResponse(content={"error": str(e)}, status_code=500)

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=3000)

Overwriting app.py


In [25]:
!mkdir -p www

In [26]:
%%writefile www/index.html
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Audio Recorder</title>
    <link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Roboto:300,400,500,700&display=swap"/>
    <link rel="stylesheet" href="https://fonts.googleapis.com/icon?family=Material+Icons"/>
    <script src="https://unpkg.com/react@18/umd/react.production.min.js"></script>
    <script src="https://unpkg.com/react-dom@18/umd/react-dom.production.min.js"></script>
    <script src="https://unpkg.com/@mui/material@5.14.5/umd/material-ui.production.min.js"></script>
    <script src="https://unpkg.com/@babel/standalone/babel.min.js"></script>
    <style>
        body { margin: 0; font-family: 'Roboto', sans-serif; }
        .container { max-width: 800px; margin: 0 auto; padding: 20px; }
        .header { background-color: #1976d2; color: white; padding: 16px; margin-bottom: 20px; }
        .chat-bubble { background-color: #f5f5f5; padding: 16px; margin-bottom: 8px; border-radius: 4px; }
        .transcription-container { max-height: 300px; overflow-y: auto; margin-top: 16px; }
        .button-container { display: flex; gap: 8px; margin-top: 16px; }
        .url-input { width: 100%; margin-bottom: 16px; }
    </style>
</head>
<body>
    <div id="root"></div>
    <script type="text/babel">
        const { useState, useRef, useEffect } = React;
        const {
            AppBar, Toolbar, Typography, Button, Container, Box, Paper, TextField
        } = MaterialUI;

        const App = () => {
            const [isRecording, setIsRecording] = useState(false);
            const [transcriptions, setTranscriptions] = useState([]);
            const [apiUrl, setApiUrl] = useState('');
            const mediaRecorderRef = useRef(null);
            const audioChunksRef = useRef([]);

            useEffect(() => {
                // 현재 URL에서 기본 API 주소 설정
                const protocol = window.location.protocol;
                const hostname = window.location.hostname;
                const port = window.location.port;
                const baseUrl = `${protocol}//${hostname}${port ? ':' + port : ''}`;
                setApiUrl(`${baseUrl}/transcribe`);
            }, []);

            const handleApiUrlChange = (event) => {
                setApiUrl(event.target.value);
            };

            const handleStartRecording = async () => {
                try {
                    const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
                    mediaRecorderRef.current = new MediaRecorder(stream);

                    mediaRecorderRef.current.ondataavailable = (event) => {
                        audioChunksRef.current.push(event.data);
                    };

                    mediaRecorderRef.current.onstop = () => {
                        sendAudioData();
                    };

                    audioChunksRef.current = [];
                    mediaRecorderRef.current.start();
                    setIsRecording(true);
                } catch (error) {
                    console.error('Error accessing microphone:', error);
                    alert('마이크 접근에 실패했습니다. 마이크 권한을 허용해주세요.');
                }
            };

            const handleStopRecording = () => {
                if (mediaRecorderRef.current && isRecording) {
                    mediaRecorderRef.current.stop();
                    setIsRecording(false);
                }
            };

            const sendAudioData = async () => {
                const audioBlob = new Blob(audioChunksRef.current, { type: 'audio/wav' });
                const formData = new FormData();
                formData.append('file', audioBlob, 'recording.wav');

                try {
                    const response = await fetch(apiUrl, {
                        method: 'POST',
                        body: formData,
                    });

                    const data = await response.json();
                    if (data.text) {
                        setTranscriptions((prev) => [...prev, data.text]);
                    } else if (data.error) {
                        setTranscriptions((prev) => [...prev, `Error: ${data.error}`]);
                    }
                } catch (error) {
                    console.error('Error sending audio data:', error);
                    setTranscriptions((prev) => [...prev, `Error: ${error.message}`]);
                }
            };

            return (
                <Container className="container">
                    <div className="header">
                        <Typography variant="h6">Audio Recorder</Typography>
                    </div>
                    <Box>
                        <TextField
                            label="API URL"
                            variant="outlined"
                            fullWidth
                            value={apiUrl}
                            onChange={handleApiUrlChange}
                            className="url-input"
                        />
                        <div className="button-container">
                            <Button
                                variant="contained"
                                color="primary"
                                onClick={handleStartRecording}
                                disabled={isRecording}
                            >
                                Start Recording
                            </Button>
                            <Button
                                variant="contained"
                                color="secondary"
                                onClick={handleStopRecording}
                                disabled={!isRecording}
                            >
                                Stop Recording
                            </Button>
                        </div>
                    </Box>
                    <div className="transcription-container">
                        {transcriptions.map((text, index) => (
                            <div key={index} className="chat-bubble">
                                {text}
                            </div>
                        ))}
                    </div>
                </Container>
            );
        };

        ReactDOM.render(<App />, document.getElementById('root'));
    </script>
</body>
</html>

Overwriting www/index.html


In [27]:
%%writefile run_server.py
import subprocess
import time
import sys
import os

# FastAPI 서버 시작
server_process = subprocess.Popen(["python", "app.py"])
print("FastAPI 서버가 시작되었습니다.")

# ngrok 실행
ngrok_process = subprocess.Popen(["./ngrok", "http", "3000"],
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE,
                                universal_newlines=True)
print("ngrok이 시작되었습니다.")

# ngrok URL 가져오기
def get_ngrok_url():
    # ngrok 상태 확인 API 호출
    result = subprocess.run(["curl", "http://localhost:4040/api/tunnels"],
                           capture_output=True,
                           text=True)
    if result.stdout:
        import json
        try:
            tunnels = json.loads(result.stdout)['tunnels']
            for tunnel in tunnels:
                if tunnel['proto'] == 'https':
                    return tunnel['public_url']
        except Exception as e:
            print(f"ngrok URL 파싱 오류: {e}")
    return None

# ngrok URL 출력
print("ngrok 터널을 설정 중입니다...")
time.sleep(5)  # ngrok이 터널을 열 시간 제공

ngrok_url = get_ngrok_url()
if ngrok_url:
    print(f"ngrok 공개 URL: {ngrok_url}")
    print(f"웹 인터페이스 접속 URL: {ngrok_url}")
    print(f"API 엔드포인트 URL: {ngrok_url}/transcribe")
else:
    print("ngrok URL을 찾을 수 없습니다. 터미널 출력을 확인하세요.")

try:
    print("\n서버를 종료하려면 Ctrl+C를 누르세요...\n")
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("서버를 종료합니다...")
    server_process.terminate()
    ngrok_process.terminate()
    print("서버가 종료되었습니다.")

Overwriting run_server.py


In [28]:
!python run_server.py

FastAPI 서버가 시작되었습니다.
ngrok이 시작되었습니다.
ngrok 터널을 설정 중입니다...
ngrok 공개 URL: https://e5b6-34-9-112-186.ngrok-free.app
웹 인터페이스 접속 URL: https://e5b6-34-9-112-186.ngrok-free.app
API 엔드포인트 URL: https://e5b6-34-9-112-186.ngrok-free.app/transcribe

서버를 종료하려면 Ctrl+C를 누르세요...

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 117MiB/s]
INFO:     Started server process [3027]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:3000 (Press CTRL+C to quit)
INFO:     175.120.109.101:0 - "GET / HTTP/1.1" 200 OK
INFO:     175.120.109.101:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
서버를 종료합니다...
서버가 종료되었습니다.
Exception ignored in: <module 'threading' from '/usr/lib/python3.11/threading.py'>
Traceback (most recent call last):
  File "/usr/lib/python3.11/threading.py", line 1568, in _shutdown
    assert tlock.locked()
           ^^^^^^^^^^^^^^
KeyboardInterrupt: 
INFO:     Shutting down
INFO:     Waiting for appl